## 一、环境配置与数据加载


In [5]:
## 1. 安装必要的库 
# ------------------------------------
# pip install transformers datasets evaluate sacrebleu torch tqdm

import torch
import evaluate
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm.auto import tqdm

# 2. 设置设备
# ------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"当前使用的设备: {device}")

# 3. 加载数据集
# ------------------------------------
# 使用 'en-zh' 配置加载英译中测试集
try:
    dataset = load_dataset("Helsinki-NLP/opus-100", "en-zh", split="test") 
    test_data = dataset.select(range(2000)) # 选取前2000对用于实验
    
    # 提取输入 (英文) 和参考 (中文)
    english_inputs = [item['translation']['en'] for item in test_data]
    chinese_references = [item['translation']['zh'] for item in test_data]
    
    print(f"成功加载 {len(english_inputs)} 条英译中数据。")

except Exception as e:
    print(f"数据加载失败，请检查网络或配置: {e}")
    
# 4. 加载模型和分词器
# ------------------------------------
model_name = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# 5. 设置T5任务前缀
# ------------------------------------
# T5模型需要一个前缀来指示任务类型
T5_PREFIX = "translate English to Chinese: "

# 设置推理参数
BATCH_SIZE = 16 
MAX_LENGTH = 128

当前使用的设备: cuda
成功加载 2000 条英译中数据。


## 二、 批量推理循环


In [6]:
## 6. 执行批量推理
# ------------------------------------
predictions = []
batch_size = BATCH_SIZE

print(f"\n开始使用 {model_name} 在 {device} 上对 {len(english_inputs)} 条数据进行推理...")

for i in tqdm(range(0, len(english_inputs), batch_size), desc="翻译进度"):
    # 获取当前批次的输入
    batch_inputs = english_inputs[i:i + batch_size]
    
    # 1. 预处理：加入前缀并分词
    tokenized_inputs = tokenizer([T5_PREFIX + input_text for input_text in batch_inputs], 
                                 return_tensors="pt", 
                                 padding=True, 
                                 truncation=True, 
                                 max_length=MAX_LENGTH)
    
    # 2. 移动到设备
    input_ids = tokenized_inputs.input_ids.to(device)
    attention_mask = tokenized_inputs.attention_mask.to(device)
    
    # 3. 生成翻译
    # 使用 num_beams=4 进行束搜索，提高翻译质量
    with torch.no_grad(): # 推理时禁用梯度计算，节省内存和时间
        outputs = model.generate(
            input_ids, 
            attention_mask=attention_mask, 
            max_length=MAX_LENGTH, 
            num_beams=4, 
            early_stopping=True
        )
    
    # 4. 后处理：解码生成的结果
    decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    # 5. 存储预测结果
    predictions.extend(decoded_preds)

print("\n批量翻译推理完成！")


开始使用 google-t5/t5-small 在 cuda 上对 2000 条数据进行推理...


翻译进度: 100%|██████████| 125/125 [02:44<00:00,  1.32s/it]


批量翻译推理完成！


## 三、评估指标计算


In [8]:
## 7. 评估翻译结果
# ------------------------------------
print("\n开始计算 BLEU 和 ROUGE 指标...")

# 准备参考译文格式：sacrebleu 要求参考是列表的列表
# 例如: [['参考1'], ['参考2'], ...]
formatted_references = [[ref] for ref in chinese_references]

# 8. 计算 BLEU 分数
# ------------------------------------
bleu_metric = evaluate.load("sacrebleu")
try:
    bleu_results = bleu_metric.compute(predictions=predictions, references=formatted_references)
    print("--- 翻译结果评估 (英译中 T5-small) ---")
    print(f"BLEU Score: {bleu_results['score']:.2f}")
except Exception as e:
    print(f"BLEU 计算失败 (可能因为预测或参考数量不匹配): {e}")

# 9. 计算 ROUGE 分数
# ------------------------------------
rouge_metric = evaluate.load("rouge")
try:
    rouge_results = rouge_metric.compute(predictions=predictions, references=chinese_references)
    print("ROUGE Results (用于摘要和句子匹配的指标):")
    
    # 打印完整的字典结构
    import json
    # 使用 indent=2 格式化打印，使其更易读
    print(json.dumps(rouge_results, indent=2)) 

except Exception as e:
    print(f"ROUGE 计算失败: {e}")


开始计算 BLEU 和 ROUGE 指标...


--- 翻译结果评估 (英译中 T5-small) ---
BLEU Score: 1.92
ROUGE Results (用于摘要和句子匹配的指标):
{
  "rouge1": 0.06681294171458776,
  "rouge2": 0.026142790232076964,
  "rougeL": 0.06121968727901171,
  "rougeLsum": 0.061102119549933526
}


## 四、随机输出翻译结果


In [9]:
# --- 查看具体的翻译内容 ---
print("\n--- 翻译结果抽查 (寻找‘有趣的’错误) ---")

# 我们选取前 5 个，以及中间的一些样本来看看
sample_indices = [0, 1, 2, 100, 500] 

for i, idx in enumerate(sample_indices):
    if idx < len(predictions):
        print(f"\n样本 {i+1} (索引 {idx}):")
        print(f"【原文 (EN)】: {english_inputs[idx]}")
        print(f"【参考 (ZH)】: {chinese_references[idx]}")
        print(f"【模型 (Pred)】: {predictions[idx]}")
        print("-" * 30)


--- 翻译结果抽查 (寻找‘有趣的’错误) ---

样本 1 (索引 0):
【原文 (EN)】: The Global Programme of Action Coordination Office, with the financial support of Belgium, is currently assisting Egypt, Nigeria, United Republic of Tanzania, Sri Lanka and Yemen to develop pilot national programmes of action for the protection of the marine environment from land-based activities.
【参考 (ZH)】: 9. 《全球行动纲领》协调处得到比利时的财政支持，目前正帮助埃及、尼日利亚、坦桑尼亚联合共和国、斯里兰卡和也门制订试行的保护海洋环境免受陆地活动影响的国家行动方案。
【模型 (Pred)】: Das Büro für Koordinierung des globalen Aktionsprogramms unterstützt derzeit mit der finanziellen Unterstützung Belgiens gyptens, Nigerias, der Vereinigten Republik Tansania, Sri Lankas und Jemen die Entwicklung von Pilotprogrammen für den Schutz der Meeresumwelt vor landwirtschaftlichen Aktivitäten.
------------------------------

样本 2 (索引 1):
【原文 (EN)】: The forum produced business contracts amounting to more than $24 million between Asian and African private companies.
【参考 (ZH)】: 该论坛达成的亚洲和非洲国家私营公司间的商业合同超过了2,400万美元。
【模型 (Pred)】: Das F